In [1]:
from SteamUser import *
import json

# Attempt to gather data of a users games via the Steam WebAPI (the documented rate limit is much higher than the actual)

In [ ]:
users = {}
userslist = []
with open("INSERT FILE NAME", "r") as file:
    for l in file.readlines():
        u = SteamUser.fromdict(json.loads(l))
        users[u.steamID] = u
    file.close()

len(users)

In [21]:
#Fix and get games

from requests import ReadTimeout, HTTPError, exceptions
from http import client

import time
from steam.webapi import WebAPI

with open("steam.key") as key:
    API = WebAPI(key.read())

In [ ]:
null_count = 0
success_count = 0
since_success = 0
for u in users.values():
    try:
        games = API.IPlayerService.GetOwnedGames_v1(
                steamid=u.steamID,
                include_appinfo=0,
                include_played_free_games=1,
                appids_filter=0,
                include_free_sub=1)['response']

        u.games = list([g['appid'] for g in games['games']])
        u.playtime = dict({a['appid']:a.get('playtime_forever') for a in games['games']})
        u.recent_playtime = dict({a['appid']:a.get('playtime_2weeks') for a in games['games']})
        u.total_playtime = sum(u.playtime.values())

        users[u.steamID] = u
        success_count += 1
        since_success = 0

    except KeyError:
        null_count += 1
        since_success += 1
    
    print("Null Count : {}, Success Count : {}, Since Success : {}, Total : {}".format(null_count, success_count, since_success, null_count + success_count) + "      ", end='\r')

In [ ]:
sum(u.games == None for u in users.values())

In [ ]:
with open("INSERT FILENAME", "w") as file:
    file.writelines(json.dumps(a.asdict()) + "\n" for a in users.values())

len(users)